**Import Libraries and Data** 

In [1]:
import numpy as np 
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/dataset/solubility_dev.csv
/kaggle/input/dataset/solubility_train.csv
/kaggle/input/dataset/solubility_test.csv


In [2]:
# import BERT tokenization

!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [3]:
import tokenization
import tensorflow as tf
import tensorflow_hub as hub
!pip install pytorch_pretrained_bert
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 123 kB 937 kB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
train_data = pd.read_csv('../input/dataset/solubility_train.csv')
test_data = pd.read_csv('../input/dataset/solubility_test.csv')

In [5]:
train_data.head()

,Meta,Sequence,Label
0,Psol_train_0,GMILKTNLFGHTYQFKSITDVLAKANEEKSGDRLAGVAAESAEERV...,1
1,Psol_train_1,MAHHHHHHMSFFRMKRRLNFVVKRGIEELWENSFLDNNVDMKKIEY...,0
2,Psol_train_2,MGSDKIHHHHHHMEKSIQDTIHGVIKLEDWMVEIVDTPQFQRLRRI...,0
3,Psol_train_3,MEKYIHSVEDYHRLISYLENNLNYEDSVVNHVIYVIAKTGMRYGEI...,0
4,Psol_train_4,MSLTDSFTVRSIEGVCFRYPLATPVVTSFGKMLNRPAVFVRVVDED...,0


**Label encoding of labels**

In [6]:
label = preprocessing.LabelEncoder()
y = label.fit_transform(train_data['Label'])
y = to_categorical(y)
print(y)

[[0. 1.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]


**Build a BERT layer**

we create a BERT embedding layer by importing the BERT model from hub.KerasLayer

In [7]:
m_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(m_url, trainable=True)

**Encoding the text**

we create a BERT vocab_file in the form a numpy array. We then set the text to lowercase and finally we pass our vocab_file and do_lower_case variables to the Tokenizer object.

In [8]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

def bert_encode(texts, tokenizer, max_len=256):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
        
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len-len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
        
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

**Build The Model**

Now we are all set to create our model. To do so, we will create a function named build_model that having tf.keras.models.Model class. Inside the function we will define our model layers. Our model will consist of three **Dense** neural network layers and also dropout layer. We have chosen a learning rate to 1e-5.

**RELU function** :- 
With default values, this returns max(x, 0), the element-wise maximum of 0 and the input tensor.
Modifying default parameters allows you to use non-zero thresholds, change the max value of the activation, and to use a non-zero multiple of the input for values below the threshold.


**Softmax function** :-
Softmax converts a real vector to a vector of categorical probabilities.
The elements of the output vector are in range (0, 1) and sum to 1.
Each vector is handled independently. The axis argument sets which axis of the input the function is applied along.
Softmax is often used as the activation for the last layer of a classification network because the result could be interpreted as a probability distribution.
The softmax of each vector x is computed as exp(x) / tf.reduce_sum(exp(x)).

**Binary corssentropy**:-
Computes the cross-entropy loss between true labels and predicted labels.
We can use this cross-entropy loss when there are only two label classes (assumed to be 0 and 1). For each example, there should be a single floating-point value per prediction.

In [9]:
def build_model(bert_layer, max_len=254):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    
    clf_output = sequence_output[:, 0, :]
    
    lay = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    lay = tf.keras.layers.Dense(32, activation='relu')(lay)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    out = tf.keras.layers.Dense(2, activation='softmax')(lay)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [10]:
max_len = 250
train_input = bert_encode(train_data.Sequence.values, tokenizer, max_len=max_len)
test_input = bert_encode(test_data.Sequence.values, tokenizer, max_len=max_len)
train_labels = y

In [11]:
labels = label.classes_
print(labels)

[0 1]


In [12]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 250)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 250)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 250)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]      

In [13]:
tf.keras.backend.clear_session()

**Run the model**

In [14]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2, verbose=1)

train_sh = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=20,
    callbacks=[checkpoint, earlystopping],
    batch_size=8,
    verbose=1
)

Epoch 1/20
6248/6248 [==============================] - ETA: 0s - loss: 0.6107 - accuracy: 0.6217
Epoch 00001: val_accuracy improved from -inf to 0.66189, saving model to model.h5
6248/6248 [==============================] - 1768s 283ms/step - loss: 0.6107 - accuracy: 0.6217 - val_loss: 0.5887 - val_accuracy: 0.6619
Epoch 2/20
6248/6248 [==============================] - ETA: 0s - loss: 0.5752 - accuracy: 0.6776
Epoch 00002: val_accuracy improved from 0.66189 to 0.68110, saving model to model.h5
6248/6248 [==============================] - 1770s 283ms/step - loss: 0.5752 - accuracy: 0.6776 - val_loss: 0.5616 - val_accuracy: 0.6811
Epoch 3/20
6248/6248 [==============================] - ETA: 0s - loss: 0.5525 - accuracy: 0.6985
Epoch 00003: val_accuracy improved from 0.68110 to 0.69518, saving model to model.h5
6248/6248 [==============================] - 1764s 282ms/step - loss: 0.5525 - accuracy: 0.6985 - val_loss: 0.5488 - val_accuracy: 0.6952
Epoch 4/20
6248/6248 [==================